# Machine Intelligence with Deep Learning
## Sampling Batches by Importance
---

In [1]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os

In [2]:
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### load the data
# if needed, specify batch sizes and shuffle settings
print('==> Loading data..')
dataloader = DataLoader()
dataloader.download_cifar()
trainloader, testloader = dataloader.get_loaders()

### Model
print('==> Building model..')
net = ResNet18()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)
if device == 'cuda':
    net.to(device)
    
### hyperparameters
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 5 # number of iterations the model gets trained
learning_rate = 1e-4 # factor for weight updates
momentum = 0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
resume = False

if resume:
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
    ckpt = torch.load('./serialized/ckpt.pth')
    best_acc, start_epoch, net = net.load(ckpt)

### Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

### Testing
def test(epoch, best_acc):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    print(acc)
    if acc > best_acc:
        print("==> Saving to checkpoint..")
        net.save(acc, epoch)
    return best_acc

==> Loading data..
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


In [3]:
for epoch in range(start_epoch, start_epoch+num_epochs):
    train(epoch)
    best_acc = test(epoch, best_acc)


Epoch: 0
32.5
==> Saving to checkpoint..

Epoch: 1
38.84
==> Saving to checkpoint..

Epoch: 2
44.2
==> Saving to checkpoint..

Epoch: 3
46.88
==> Saving to checkpoint..

Epoch: 4
49.92
==> Saving to checkpoint..
